In [1]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.5 MB/s eta 0:

In [4]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00


In [5]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [77]:
from transformers import AutoConfig
id2label = {
  "0": "Negative",
  "1": "Neutral",
  "2": "Positive"
}

In [85]:
from transformers import AutoModel, AutoTokenizer

model_path = "Henok21/test_trainer"

config = AutoConfig.from_pretrained(model_path , id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained(model_path , config  = config)

In [79]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [80]:
text = "medicen is best!"
text = preprocess(text)
text

'medicen is best!'

In [81]:
# PyTorch-based models

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

array([0.09499077, 0.49290192, 0.4121073 ], dtype=float32)

In [82]:
config.id2label = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}

In [87]:
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

KeyError: ignored

In [88]:
model_path = "Henok21/test_trainer"

# Gradio App

In [89]:
import gradio as gr

In [94]:
def sentiment_analysis(text):
  text = preprocess(text)

  # PyTorch-based models
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores_ = output[0][0].detach().numpy()
  scores_ = softmax(scores_)

  labels = ["Negative" , "Neutral" , "Positive"]
  scores = {l : float(s) for (l , s) in zip(labels , scores_)}

  return scores


In [93]:
# Import the pipeline class
from transformers import pipeline

# Create a model
model = model

# Create a tokenizer
tokenizer = tokenizer

# Create a pipeline object for sentiment analysis using your model and tokenizer
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer,  config = config)

# Use the pipeline object on some input text
result = nlp("I love this movie!")

# Print the result
print(result)


KeyError: ignored

In [74]:
config.label2id = {
    "Negative" : 0 ,
    "Neutral" : 1 ,
    "Positive" : 2
}

In [95]:
sentiment_analysis("covid is good")

{'Negative': 0.11060222238302231,
 'Neutral': 0.4763108193874359,
 'Positive': 0.4130869507789612}

In [48]:
demo = gr.Interface(
    fn = sentiment_analysis,
    inputs = gr.Textbox(placeholder = "write down"),
    outputs = "text",
    interpretation = "default",
    examples = [
        ["whats up with the vaccine"],
        ["i don't like it"]
    ]
)

In [49]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>